In [1]:
import pandas as pd
import os

In [2]:
df=pd.read_excel('spamhamV2.xlsx', sep=',')

In [3]:
df['Email']=df.Email.str.replace(':','')
df['Email']=df.Email.str.replace('"','')
df['Email']=df.Email.str.replace("'","")
df['Email']=df.Email.str.replace('(','')
df['Email']=df.Email.str.replace(')','')
df['Email']=df.Email.str.replace('\n','')
df['Email']=df.Email.str.replace(',','')
df['Email']=df.Email.str.replace('.','')
df['Email']=df.Email.str.lower()


In [4]:
df['Email']=df['Email'].str.split(pat=None,n=-1,expand=False)


In [5]:
dfTreinamento=df.sample(n=None, frac=0.75, replace=False, weights=None, random_state=None, axis=None)
dfTeste=df.sample(n=None, frac=0.25, replace=False, weights=None, random_state=None, axis=None)

In [6]:
dfTreinamento.to_csv('var_Treinamento.csv')
dfTeste.to_csv('var_Teste.csv')

In [7]:
#Calculando a probabilidade de ser Spam dentro do dataframe de Treinamento
Spam=dfTreinamento[dfTreinamento.Class=='spam']
Pspam=len(Spam)/len(dfTreinamento['Class'])
Pspam

0.13137114142139267

In [8]:
#Calculando a probabilidade de ser Ham dentro do dataframe de Treinamento
Ham=dfTreinamento[dfTreinamento.Class=='ham']
Pham=1-Pspam
Pham

0.8686288585786073

In [9]:
dfTotal=dfTreinamento['Email'].sum(axis=None, skipna=None, level=None, numeric_only=None)
serie = pd.Series(dfTotal)
palavras = serie.value_counts()
#j=len(dfTotal)

In [10]:
#Calculando a probabilidade de uma palavra acontecer se a mensagem na base treinamento é considerada SPAM
dfTreiSpam=dfTreinamento[dfTreinamento.Class=='spam']
dfTreiSpam=dfTreiSpam['Email'].sum(axis=None, skipna=None, level=None, numeric_only=None)
SpamSeries=pd.Series(dfTreiSpam)

PS=SpamSeries.value_counts()

#j=len(Spam)+len(Ham)
PalavrasSpam=(PS+1)/(len(SpamSeries)+len(palavras))
#PalavrasSpam

In [11]:
#Calculando a probabilidade de uma palavra acontecer se a mensagem na base treinamento é considerada Ham
dfTreiHam=dfTreinamento[dfTreinamento.Class=='ham']
dfTreiHam=dfTreiHam['Email'].sum(axis=None, skipna=None, level=None, numeric_only=None)
HamSeries=pd.Series(dfTreiHam)

PH=HamSeries.value_counts()

PalavrasHam=(PH+1)/(len(HamSeries)+len(palavras))
#PalavrasHam

In [12]:
def Bayes(email):   
    var_s=Pspam
    var_h=Pham
    
    for i in email:
        if i in (PalavrasHam):
            var_h*=PalavrasHam[i]
        else:
            var_h*=1/(len(palavras))
    
    for i in email:
        if i in PalavrasSpam:
            var_s*=PalavrasSpam[i]
        else:
            var_s*=1/(len(palavras))
    
    if var_s > var_h:
        return("spam")
    else:
        return("ham")

In [13]:
def Tester(Teste):
    frase = Teste.Email
    return Bayes(frase)

T = dfTeste.apply(Tester,axis=1)
dfTeste["Resultados"] = T
dfTeste

,Email,Class,Resultados
1057,"[ard, 515, like, dat, y?]",ham,ham
3137,"[wat, r, u, doing, now?]",ham,ham
3052,"[pls, send, me, the, correct, name, da]",ham,ham
2328,"[am, surfing, online, store, for, offers, do, ...",ham,ham
5122,"[\not, enufcredeit, tocallshall, ileave, uni, ...",ham,spam
940,"[no, we, sell, it, all, so, well, have, tons, ...",ham,ham
2221,"[great, i, was, getting, worried, about, you, ...",ham,ham
965,"[or, better, still, can, you, catch, her, and,...",ham,ham
838,"[we, tried, to, contact, you, re, our, offer, ...",spam,spam
5308,"[im, coming, back, on, thursday, yay, is, it, ...",ham,ham


In [17]:
dfTeste['Resultados'].value_counts()

ham     1211
spam     182
Name: Resultados, dtype: int64

In [18]:
dfTeste['Class'].value_counts()

ham     1208
spam     185
Name: Class, dtype: int64